In [ ]:
#IMPORTS put all here
import random
import matplotlib.pyplot as plt

#tf imports for reading file
import numpy as np
import os
import PIL

import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization


#extras
print(tf.__version__)
import pathlib
import shutil

In [ ]:
#path to image dataset
spec_path = 'content/spectrograms6sec'

genre_list=os.listdir(spec_path)
genre_list


In [ ]:
#resets the experiment
%rm -rf content/spectrograms6secnew

In [ ]:
os.mkdir('content/spectrograms6secnew')
dest_path='content/spectrograms6secnew'

for genre in genre_list:
  os.mkdir(f'content/spectrograms6secnew/{genre}')

In [ ]:


#no of samples per class
n=3000
cnt=0

random.seed(123)

for genre in genre_list:
  genre_path=os.path.join(spec_path,genre)
  dest_genre_path=os.path.join(dest_path,genre)

  #shuffling and selecting 3000 samples
  genre_songs=os.listdir(genre_path)
  random.shuffle(genre_songs)
  selected_spec=genre_songs[:n]
  
  for song in selected_spec:
      source=os.path.join(genre_path,song)
      dest=os.path.join(dest_genre_path,song)
      d= shutil.copyfile(source, dest)




In [ ]:
data_dir=pathlib.Path(dest_path)
data_dir

In [ ]:
#adding dataset
batch_size=64
image_height=75
image_width=150


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)


In [ ]:
class_names = train_ds.class_names
print(class_names)

import matplotlib.pyplot as plt

plt.figure(figsize=(45, 15))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")


In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = Sequential()

In [ ]:
num_classes = 10

# 16 - number of filters
# (3,3) - filter size
# 1 - stride of filter
# relu activation - converts negative numbers to 0 (rectified linear activation)
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(75,150,3)))
BatchNormalization()
# condenses data 
model.add(MaxPooling2D())

#model.add(Conv2D(32, (3,3), 1, activation='relu' ))
#BatchNormalization()
#model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu' ))
BatchNormalization()
model.add(MaxPooling2D())

# condensing into single output
model.add(Flatten())

model.add(Dense(256, activation='relu'))
BatchNormalization()
model.add(Dropout(rate=0.2))
# softmax activation - maps to a multiclass classification
model.add(Dense(1, activation='softmax'))

In [ ]:
model.compile('adam',loss=tf.losses.BinaryCrossentropy(),metrics=['accuracy'])
model.summary()

In [ ]:
logdir='logs'
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)
early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

In [ ]:
hist=model.fit(train_ds,epochs=100,validation_data=val_ds,callbacks=[tensorboard_callback,early])